Antonio Iorio s317748 `<s317748@studenti.polit.it>`\
[`https://github.com/antonioior/Computational_Intelligence.git`](https://github.com/antonioior/Computational_Intelligence.git)\
Development Lab 01 with Alessia Intini s309895   

In [1]:
from random import random
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue
from math import ceil

import numpy as np

We have decided to use a fixed sets of value to campare the number of steps to resolve the set-covering across the 3 different h function. Because when we use a random values we can't compare the efficiency of the every h functions.
The fixed sets used is: 

```python
PROBLEM_SIZE = 10
NUM_SETS = 20
SETS = (np.array([False,  True, False,  True, False, False, False, False, False,
       False]), np.array([ True,  True, False, False, False, False, False,  True, False,
        True]), np.array([ True, False,  True, False,  True, False, False, False,  True,
       False]), np.array([False,  True, False, False, False, False, False,  True, False,
        True]), np.array([ True,  True,  True,  True, False, False, False,  True, False,
       False]), np.array([False, False, False,  True, False, False, False, False, False,
       False]), np.array([False, False, False, False, False, False,  True, False, False,
       False]), np.array([ True, False,  True, False, False, False, False, False, False,
       False]), np.array([False,  True,  True,  True,  True, False, False, False,  True,
       False]), np.array([False, False, False, False, False, False, False, False, False,
       False]), np.array([False, False,  True, False, False, False,  True, False, False,
        True]), np.array([False,  True,  True, False, False,  True, False, False,  True,
       False]), np.array([False, False,  True, False, False, False, False, False,  True,
        True]), np.array([False, False,  True, False,  True, False, False, False, False,
       False]), np.array([ True,  True,  True,  True, False, False,  True, False, False,
        True]), np.array([False, False, False, False, False, False,  True, False, False,
        True]), np.array([ True,  True,  True, False, False,  True, False, False, False,
       False]), np.array([False,  True,  True, False, False, False, False, False,  True,
       False]), np.array([ True,  True, False,  True, False, False,  True, False, False,
       False]), np.array([False, False, False,  True, False, False,  True,  True, False,
        True]))
```

In [2]:
PROBLEM_SIZE = 20       #dimensione ogni elemento tupla
NUM_SETS = 50           #numero di elementi della tupla
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
# SETS = (np.array([False,  True, False,  True, False, False, False, False, False,
#        False]), np.array([ True,  True, False, False, False, False, False,  True, False,
#         True]), np.array([ True, False,  True, False,  True, False, False, False,  True,
#        False]), np.array([False,  True, False, False, False, False, False,  True, False,
#         True]), np.array([ True,  True,  True,  True, False, False, False,  True, False,
#        False]), np.array([False, False, False,  True, False, False, False, False, False,
#        False]), np.array([False, False, False, False, False, False,  True, False, False,
#        False]), np.array([ True, False,  True, False, False, False, False, False, False,
#        False]), np.array([False,  True,  True,  True,  True, False, False, False,  True,
#        False]), np.array([False, False, False, False, False, False, False, False, False,
#        False]), np.array([False, False,  True, False, False, False,  True, False, False,
#         True]), np.array([False,  True,  True, False, False,  True, False, False,  True,
#        False]), np.array([False, False,  True, False, False, False, False, False,  True,
#         True]), np.array([False, False,  True, False,  True, False, False, False, False,
#        False]), np.array([ True,  True,  True,  True, False, False,  True, False, False,
#         True]), np.array([False, False, False, False, False, False,  True, False, False,
#         True]), np.array([ True,  True,  True, False, False,  True, False, False, False,
#        False]), np.array([False,  True,  True, False, False, False, False, False,  True,
#        False]), np.array([ True,  True, False,  True, False, False,  True, False, False,
#        False]), np.array([False, False, False,  True, False, False,  True,  True, False,
#         True]))
State = namedtuple('State', ['taken', 'not_taken'])
print(SETS)

(array([False, False, False, False, False,  True, False,  True, False,
        True,  True, False, False,  True,  True,  True, False,  True,
       False, False]), array([ True, False, False, False, False,  True, False, False, False,
        True, False,  True, False,  True,  True, False,  True,  True,
        True, False]), array([False,  True, False, False,  True, False, False, False, False,
       False,  True, False, False, False, False, False, False,  True,
       False,  True]), array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False]), array([ True, False,  True,  True, False,  True,  True, False,  True,
       False, False,  True,  True, False, False,  True, False, False,
       False, False]), array([False,  True,  True,  True, False,  True,  True, False, False,
        True, False,  True,  True, False,  True, False,  True,  True,
       False, False]), array([False, False, 

In [3]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

In [4]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [5]:
def h1(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def distance(state):
    return len(state.taken) + h3(state)

Results of execution are:
 - h1 resolves it in 85 steps
 - h2 resolves it in 9 steps
 - h3 resolves it in 9 steps

In [6]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((distance(new_state), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 3 steps (2 tiles)


In [7]:
current_state

State(taken={42, 5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49})